## Example: Initialization and Primitives of the Executor

This example shows different ways to initialize the Executor und run various jobs.

In [1]:
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Estimator as RuntimeEstimator

from squlearn.util import Executor

The following cell shows different ways to initialize of the ``Executor`` class from different backends or services.

In [2]:
# from a string representing the simple Qiskit Aer simulators:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")
# from a backend following the Qiskit backend standard:
executor = Executor(Aer.get_backend("statevector_simulator"))

In [3]:
# from a backend obtained from the Qiskit IBM runtime service (here, the account has to be set-up previously):
service = QiskitRuntimeService(channel="ibm_quantum")
# Alternative: service = QiskitRuntimeService(channel="ibm_quantum",token="YOUR_TOKEN_HERE")
executor = Executor(service.get_backend("ibm_nairobi"))
# from a session set-up with the Qiskit IBM runtime service:
session = Session(service, backend=service.get_backend("ibm_nairobi"), max_time=28800)
executor = Executor(session)

# from the Qiskit IBM runtime Estimator primitive:
session = Session(service, backend=service.get_backend("ibm_nairobi"), max_time=28800)
estimator = RuntimeEstimator(session=session)
executor = Executor(estimator)

In [4]:
# from a Qiskit simulator primitive:
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("qasm_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("qasm_simulator")))

The cells demonstrates how to set the number of shots utilized in the circuit evaluation.

In [5]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())
# Set shots
executor.set_shots(1234)
print("Adjusted shots:", executor.get_shots())
# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots:", executor.get_shots())

Current shots as set before: 1024
Adjusted shots: 1234
Reset shots: 1024


In this cell, we calculate an expectation value using the Estimator primitive, which is accessible through the ``Executor`` class. The executor generates modified Primitives with enhanced functionality, including caching, automatic session management, and logging capabilities. These modified primitives can be seamlessly incorporated into your workflow or other Qiskit routines.

In [6]:
# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate an observable:
observable = SparsePauliOp("ZI")

# Get the Executor Estimator Primitive and call run:
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())

# Get the Executor Sampler Primitive and call run:
sampler = executor.get_sampler()
print(sampler.run(circuit.measure_all(inplace=False)).result())

EstimatorResult(values=array([-0.10483684]), metadata=[{'variance': 0.9890092362820049, 'shots': 4321}])
SamplerResult(quasi_dists=[{0: 0.380859375, 3: 0.5302734375, 2: 0.0498046875, 1: 0.0390625}], metadata=[{'shots': 1024}])


The executor can also be used to execute ``backend.run()``. However, caching is not yet implemented for this case.

In [7]:
job = executor.backend_run(circuit)
job.result()

Result(backend_name='qasm_simulator', backend_version='0.12.0', qobj_id='', job_id='489c4c9d-b871-4058-a073-627517940b6b', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, metadata=None, n_qubits=2, name='circuit-163', qreg_sizes=[['q', 2]]), status=DONE, seed_simulator=3039238814, metadata={'batched_shots_optimization': False, 'method': 'stabilizer', 'active_input_qubits': [], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 0, 'num_clbits': 0, 'input_qubit_map': [], 'measure_sampling': False, 'parallel_shots': 1, 'parallel_state_update': 12}, time_taken=1.01e-05)], date=2023-11-10T13:16:03.897375, status=COMPLETED, header=None, metadata={'parallel_experiments': 1, 'omp_enabled': True, 'max_memory_mb': 16135, 'max_gpu_memory_mb': 0, 'num_processes_per_experiments': 1, 'mpi_rank': 0, 'num_mpi_processes': 1, 'time_taken_execute': 6.5e-05}, time

Additionally, the executor maintains a detailed log of background operations. This feature proves especially valuable when optimizing real backends, allowing you to gain insights into the underlying processes and activities.

In [8]:
executor = Executor("qasm_simulator", log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()

print(estimator.run(circuit, observable, shots=4321).result())

EstimatorResult(values=array([-0.12566536]), metadata=[{'variance': 0.9842082184919416, 'shots': 4321}])


The executor has a cache where it stores and can reuse job results. In this example, we change the number of shots to tell apart the first and second runs of the same job, and both runs are stored in separate caches. However, the third job simply reuses the cached result from its first execution.

In [9]:
executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(4321)
estimator = executor.get_estimator()
print(estimator.run(circuit, observable).result())
executor.set_shots(1234)
print(estimator.run(circuit, observable).result())
# This one is load from the cached and not executed again
executor.set_shots(4321)
print(estimator.run(circuit, observable).result())

EstimatorResult(values=array([-0.13260819]), metadata=[{'variance': 0.982415067269147, 'shots': 4321}])
EstimatorResult(values=array([-0.11345219]), metadata=[{'variance': 0.9871286010365417, 'shots': 1234}])
EstimatorResult(values=array([-0.13260819]), metadata=[{'variance': 0.982415067269147, 'shots': 4321}])
